In [14]:
import numpy as np
import pandas as pd
import encode
import allel
import pyensembl

In [2]:
all_snps = encode.read_file('IBD_all_snps.txt')

In [3]:
all_snps = all_snps[['rs' in val for val in all_snps.SNP]]

In [8]:
sim_snp_info = all_snps[all_snps.SNP.isin(sim.columns[:-1])]

In [9]:
sim_snp_info.head()

,CHR,SNP,BP,A1,A2,FRQ_A_12882,FRQ_U_21770
3524,10,rs12261181,1806788,T,C,0.118,0.118
5802,10,rs10751837,2117889,T,C,0.0337,0.0303
5883,10,rs147523225,2979554,A,C,0.00783,0.00611
7886,10,rs59231667,1633472,T,C,0.99,0.993
9404,10,rs12262213,2788331,T,C,0.99,0.993


# Additive Encoding and Fst
- get_fst_snps takes the summary statistics for IBD and creates an additive encoding by simulating genotypes of donors.
- Fst is also computed for the simulated data. This Fst is used to filter which SNPs to consider.
- Below is the simulated genotypes filtered by and Fst of 0.001.
- 0 represents AA: the homozygous A2 (non-risk allele)
- 1 represents AB: the heterozygous A1 (risk and non-risk allele)
- 2 represents BB: the homozygous A1 (risk allele)
- Each value is computed by sampling from a binomial where the probability of "success" is the probability of A1 occurring for the corresponding SNP.
- Cases and controls were determined by using the probability of A1 in the affected population and the probability of A1 in the unaffected population.

In [5]:
sim = pd.read_csv('full_sim_fst0.001.csv')

In [6]:
sim = sim[sim.columns[1:]]

In [7]:
sim.head()

,rs79043624,rs114499252,rs946159,rs12401349,rs1320601,rs9730623,rs192741850,rs12034861,rs185312257,rs12126235,...,rs2375690,rs146717347,rs77054243,rs16934283,rs11139605,rs149876952,rs147845411,rs4878002,rs186169169,Type
0,2,2,0,2,0,2,2,0,2,2,...,2,2,2,2,1,2,0,0,0,CASE
1,2,2,0,2,0,0,2,0,2,1,...,2,2,2,2,0,2,0,0,0,CASE
2,2,2,0,2,0,0,2,0,2,2,...,2,2,2,1,0,2,0,0,0,CASE
3,2,2,0,2,0,0,2,0,2,2,...,2,2,2,2,0,2,0,0,0,CASE
4,2,2,0,2,0,0,2,0,2,2,...,2,2,2,2,1,2,0,0,0,CASE


In [4]:
#fst = encode.compute_fst(sim)

# Genotypic Encoding
- This encoding builds off of the simutation obtained from the additive approach (above).
- This encoding scheme treats each genotype equally by creating three categories for each SNP (AA, AB, BB) and using a one-hot vector to represent the genotype for a SNP.

In [5]:
#gt_enc = encode.genotype_encode(sim)

In [13]:
#gt_enc.to_csv('gt_enc.fst0.001.csv')
gt_enc = pd.read_csv('gt_enc.fst0.001.csv')

In [3]:
gt_enc = gt_enc[gt_enc.columns[1:]]

In [7]:
gt_enc.head()

,rs79043624_AA,rs79043624_AB,rs79043624_BB,rs114499252_AA,rs114499252_AB,rs114499252_BB,rs946159_AA,rs946159_AB,rs946159_BB,rs12401349_AA,...,rs147845411_AA,rs147845411_AB,rs147845411_BB,rs4878002_AA,rs4878002_AB,rs4878002_BB,rs186169169_AA,rs186169169_AB,rs186169169_BB,Type
0,0,0,1,0,0,1,1,0,0,0,...,1,0,0,1,0,0,1,0,0,CASE
1,0,0,1,0,0,1,1,0,0,0,...,1,0,0,1,0,0,1,0,0,CASE
2,0,0,1,0,0,1,1,0,0,0,...,1,0,0,1,0,0,1,0,0,CASE
3,0,0,1,0,0,1,1,0,0,0,...,1,0,0,1,0,0,1,0,0,CASE
4,0,0,1,0,0,1,1,0,0,0,...,1,0,0,1,0,0,1,0,0,CASE


In [5]:
gt_enc.shape

(20000, 6973)

# Recessive/Dominant Encoding
- This encoding approach indicates when a SNP has at least one copy of an allele.
- For example, for SNP rs79043624 for patient 0 we can see that A is 0 and B is 1. This means that this patient as at least one copy of the risk allele and no copies of the other allele. If we were to translate this into a genotype it would be BB.
- As follows, the genotype AB would be translated into [1, 1] and AA would be [1, 0].

In [14]:
rec_enc = pd.read_csv('rec_enc_fst0.001.csv')
rec_enc = rec_enc[rec_enc.columns[1:]]

In [15]:
rec_enc.head()

,rs79043624_A,rs79043624_B,rs114499252_A,rs114499252_B,rs946159_A,rs946159_B,rs12401349_A,rs12401349_B,rs1320601_A,rs1320601_B,...,rs11139605_B,rs149876952_A,rs149876952_B,rs147845411_A,rs147845411_B,rs4878002_A,rs4878002_B,rs186169169_A,rs186169169_B,Type
0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,...,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,CASE
1,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,CASE
2,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,CASE
3,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,CASE
4,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,...,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,CASE


In [16]:
rec_enc.shape

(20000, 9803)

In [6]:
#rec_enc = encode.rec_dom_encode(gt_enc)

In [10]:
#rec_enc['Type'] = gt_enc['Type']

In [12]:
#rec_enc.to_csv('rec_enc_fst0.001.csv')

# Map SNPs to Gene info

In [15]:
ensembl = pyensembl.EnsemblRelease()

In [17]:
cols = ['snp_id', 'gene_id', 'gene_name', 'biotype', 'contig', 'start', 'end', 'strand']
genes_df = pd.DataFrame(columns=cols)

for row in sim_snp_info.iterrows():
    snp = row[1].SNP
    if row[1].BP == None: break
    pos = int(row[1].BP)
    chro = row[1].CHR
    genes = ensembl.genes_at_locus(contig=chro, position=pos)
    
    if len(genes) > 0:
        rows = np.array([[snp, 
                         gene.gene_id, 
                         gene.gene_name, 
                         gene.biotype, 
                         gene.contig, 
                         gene.start, 
                         gene.end,
                         gene.strand] for gene in genes])
             
        genes_df = pd.concat([genes_df, pd.DataFrame(rows, columns=genes_df.columns)])

In [18]:
genes_df

,snp_id,gene_id,gene_name,biotype,contig,start,end,strand
0,rs59231667,ENSG00000185736,ADARB2,protein_coding,10,1177313,1737525,-
0,rs2296443,ENSG00000157881,PANK4,protein_coding,1,2508537,2526611,-
0,rs881640,ENSG00000157881,PANK4,protein_coding,1,2508537,2526611,-
0,rs181701000,ENSG00000187730,GABRD,protein_coding,1,2019329,2030758,+
0,rs151148112,ENSG00000215267,AKR1C7P,transcribed_unprocessed_pseudogene,10,5275173,5288470,-
...,...,...,...,...,...,...,...,...
0,rs73669822,ENSG00000187239,FNBP1,protein_coding,9,129887187,130043194,-
0,rs149275167,ENSG00000165695,AK8,protein_coding,9,132725578,132878777,-
0,rs149381252,ENSG00000186350,RXRA,protein_coding,9,134317098,134440585,+
0,rs111369550,ENSG00000130558,OLFM1,protein_coding,9,135075422,135121179,+


In [19]:
genes_df.to_csv('full_gene_snp_map.csv')